In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException, ElementNotInteractableException, NoSuchElementException
import time
import re
import json
from wakepy import keepawake
import pandas as pd

In [2]:
# Initialize the df to store all of our data
df = pd.DataFrame(columns=["Store", "Item Name", "Price", "Units", "Unit Price", "Amount", "Tags", "In Stock", "Image"])

In [3]:
# Get the image representing the object
def get_image(ele):
    src_value = ""
    try:
        img_tag = ele.find_element(By.CSS_SELECTOR, 'img')
        src_value = img_tag.get_attribute('srcset')
    except NoSuchElementException:
        src_value = ""
        
    return src_value

# Get the text data that stores name, price, etc.
def get_text_data(ele):
    # Initialize the data to return as empty strings. If the desired data doesn't exist,
    # return nothing for it.
    text_list = []
    unit_price = ""
    individual_weight = ""
    stock = ""
    
    try:
        # Find all leaf-node spans (no children spans)
        text_data = ele.find_elements(By.CSS_SELECTOR, 'span:not(:has(span))')

        # Filter by length > 2. For some reason, price is stored as "$0.18",
        # and then also "$", "0", "1", "8", so we want to ignore these
        for span in text_data:
            html = span.text
            if len(html) > 2:
                text_list.append(html)                    

        # Get the unit price. Not all items have this attribute
        try:
            unit_price = ele.find_element(By.CSS_SELECTOR, 'div[title]').text
        except NoSuchElementException:
            unit_price = ""

            
        # There are at most 4 text objects below the item, stored in divs.
        # The bottom-most one will tell us the weight. We start at 4 (the lowest)
        # which will give us the "in stock" information (if any).
        # So long as "stock" isn't mentioned, it's the weight, and we can exit the loop.
        k = 4
        while k > 0:
            try:
                # Try to find the element with the specified CSS selector
                individual_weight_element = ele.find_element(By.CSS_SELECTOR, f'div[aria-label="Product"] a div:nth-child(2) div:nth-child({k})')

                if 'stock' in individual_weight_element.text:
                    stock = individual_weight_element.text
                    k -= 1
                    continue


                else:
                    individual_weight = individual_weight_element.text
                    k = 0
            except NoSuchElementException:
                # If not found, set individual_weight to an empty string
                individual_weight = ""
                k -= 1

    except NoSuchElementException:
        return text_list, unit_price, individual_weight, stock
        
    return text_list, unit_price, individual_weight, stock

# Scroll to the desired carousel within the web page and click on the button
# to expand the listed items.
def scroll_to_carousel_and_click(car):
    # Locate the desired div
    first_div_child = car.find_element(By.CSS_SELECTOR, 'div:nth-child(1)')
    second_div = first_div_child.find_element(By.CSS_SELECTOR, 'div:nth-child(2)')
    
    # Scroll to the div containing our carousel of interest
    ActionChains(driver).move_to_element(second_div).perform()

    # Expand the items so we can view them all
    first_button = second_div.find_element(By.CSS_SELECTOR, 'button:nth-child(1)')
    first_button.click()
    
# Scroll to the bottom of the given modal
def scroll_to_bottom_of_modal(mod):
    # Scrolling to the bottom may take multiple tries as elements load slowly.
    # Two scrolls works for now but may need to add more
    driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", mod)
    time.sleep(1)
    driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", mod)
    
# Parse the text into a useful format.
def parse_text_data(text_data_list, src_data):
    # Initialize the data to return as empty strings. If the desired data doesn't exist,
    # return nothing for it.
    tags = []
    price = ""
    name = ""
    units = ""

    # The src is a list of links. We just get the first one.
    src = src_value.split()
    if len(src) != 0:
        src = src[0]
    else:
        src = ""

    # Remove empty strings.
    text_data_list = [element for element in text_data_list if element.strip() != ""]

    # The price will be the only element to start with "$". This line finds the index of that element
    price_index = next(i for i, element in enumerate(text_data_list) if element.startswith("$"))
    
    try:
        # If there are units for the item, it starts with a '/' or 'each'. Not all items have this. But, if
        # it exists, we store it and then remove it from the list of data.
        units_index = next(i for i, element in enumerate(text_data_list) if element.startswith("/") or element.startswith("each"))
        units = text_data_list.pop(units_index)
    except StopIteration:
        units_index = None 

    # Loop through the text data
    for i in range(len(text_data_list)):
        # If an item is before the price index, it is a "tag". Things like "organic", "non-gmo"
        if i < price_index:
            tags.append(text_data_list[i])
        #  If an item is at the price index, store it as the price
        elif i == price_index:
            price = text_data_list[i]
        # Whatever is right after the price will be the item name
        elif i == price_index + 1:
            name = text_data_list[i]
            
    return tags, price, name, units, src

In [4]:
# driver = webdriver.Chrome(ChromeDriverManager().install())
driver = webdriver.Chrome(executable_path='/Users/jakesanghavi/Downloads/chromedriver-mac-arm64/chromedriver')
driver.maximize_window()

# Open Instacart to your desired store
driver.get('https://www.instacart.com/aldi')

wait = WebDriverWait(driver, 5)  # Maximum wait time of 5 seconds

In [5]:
# Remove modals that block website
elements_to_delete = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'ReactModalPortal')))

for element in elements_to_delete:
    driver.execute_script("arguments[0].remove();", element)

# This allows us to scroll and click again
body_element = driver.find_element_by_tag_name('body')
driver.execute_script("arguments[0].style.overflow = 'visible';", body_element)

In [6]:
# Function to scroll slowly to the bottom
def slow_scroll_to_bottom():
    actions = ActionChains(driver)
    t = 0
    
    # If it's been longer than 10 seconds, something has likely gone wrong.
    # So, cut off the scroll early and go back to the top
    while t < 10:
        actions.send_keys(Keys.PAGE_DOWN).perform()
        # Adjust the sleep time based on your needs
        time.sleep(1)
        # Check if you've reached the bottom of the page
        if driver.execute_script("return (window.innerHeight + window.scrollY) >= document.body.scrollHeight;"):
            break
        
        t += 1

# Scroll slowly to the bottom
slow_scroll_to_bottom()

# Wait 2 seconds to allow for final loading
time.sleep(2)

# Find all elements with aria-label="item carousel"
# These are the product groupings we want to jump into
carousels = driver.find_elements(By.CSS_SELECTOR, '[aria-label="item carousel"]')

In [7]:
# These divs contain the buttons we need to click on within each item carousel
reakit_portals = driver.find_elements(By.CLASS_NAME, '__reakit-portal')

# List of dataframes to be concatenated later.
to_concat = []

# We want to start from the second reakit-portal. The first one is not useful.
# We will then iterate through each of the associated carousels.
j = 1

# Iterate through each div
for carousel in carousels:
    # Scroll to where the carousel is located and click to expand its tems
    scroll_to_carousel_and_click(carousel)
    
    # Set a variable equal to the modal that opens
    modal = reakit_portals[j].find_element(By.CSS_SELECTOR, 'div:nth-child(1) div:nth-child(1) div:nth-child(2)')
        
    # Scroll to the bottom of the modal to load all of its elements
    # Careful here: may not load all products. We may have to add slower scrolling
    scroll_to_bottom_of_modal(modal)

    # Get all li objects from the list in the modal. These are the grocery items.
    product_list = modal.find_elements(By.CSS_SELECTOR, 'div:nth-child(1) ul li')
    
    # Iterate through the listed products
    for product in product_list:
        # Get the name of the item
        name = product.find_element(By.CSS_SELECTOR, 'div div div a').get_attribute('href')
        
        # Get the image representing the item
        src_value = get_image(product)
              
        # Get the other textual data about the item
        text_list, unit_price, individual_weight, stock = get_text_data(product)

        # Parse the text data into useful data
        tags, price, name, units, src = parse_text_data(text_list, src_value)

        # Add our dataframe of data to a list of dataframes. These will be concatenated to the full df later.
        cols = df.columns
        row = pd.DataFrame(data=[["Aldi", name, price, units, unit_price, individual_weight, tags, stock, src]], columns=cols)

        to_concat.append(row)
        
        j += 1
    
# Process
## In the text_data, remove data like 'starts with each' or 'Add'.
## Tags are whatever come before $
## Price is whatever starts with $
# Item name is whatever comes right after $
# Amount or unit price is whatever comes right after Name
# Make sure to ignore empty strings

ElementClickInterceptedException: Message: element click intercepted: Element <button type="button" aria-haspopup="dialog" class="e-1bfzg3z">...</button> is not clickable at point (1240, 556). Other element would receive the click: <div class="e-13udsys">...</div>
  (Session info: chrome=121.0.6167.139)


In [ ]:
# Close the browser
driver.quit()

In [8]:
# Add all of our results to the full dataframe
df = pd.concat([df] + to_concat, ignore_index=True)

# Sometime there is no unit price, but it gets set anyways
# It always gets set the the amount/weight in this case
# So, if these two are the same, then set unit price 
# to be an empty string
mask = df['Unit Price'] == df['Amount']
df.loc[mask, 'Unit Price'] = ""
df

,Store,Item Name,Price,Units,Unit Price,Amount,Tags,In Stock,Image
0,Aldi,Bananas,$0.18,each (est.),$0.49 / lb,About 0.36 lb each,[],,https://www.instacart.com/image-server/257x257...
1,Aldi,"Strawberries, Package",$3.25,,,1 lb,[],Many in stock,https://www.instacart.com/image-server/257x257...
2,Aldi,Blueberries Package,$3.55,,,1 pint container,[],Many in stock,https://www.instacart.com/image-server/257x257...
3,Aldi,"Mandarins, Bag",$3.09,,,3 lb,[In Season],Many in stock,https://www.instacart.com/image-server/257x257...
4,Aldi,Organic Bananas,$0.29,each (est.),$0.75 / lb,About 0.39 lb each,[Organic],,https://www.instacart.com/image-server/257x257...
5,Aldi,Raspberries Package,$2.75,,,6 oz Container,[],Many in stock,https://www.instacart.com/image-server/257x257...
6,Aldi,"Mission Mini Avocados, Bag",$2.55,,,6 ct,[],Many in stock,https://www.instacart.com/image-server/257x257...
7,Aldi,Red Grapes,$4.40,/pkg (est.),$2.19 / lb,About 2.01 lb / package,[],,https://www.instacart.com/image-server/257x257...
8,Aldi,Blackberries,$2.45,,,6 oz container,[],Many in stock,https://www.instacart.com/image-server/257x257...
9,Aldi,"Honeycrisp Apples, Bag",$3.65,,,2 lb,[],,https://www.instacart.com/image-server/257x257...


In [ ]:
# Save the dataframe to csv
df.to_csv("instacart_scraper_v1_results.csv")